In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import seaborn as sns
import os
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import imodelsx.process_results
import qa_questions
import random
import joblib
from tqdm import tqdm
from collections import defaultdict
import feature_spaces
fit_encoding = __import__('01_fit_encoding')
import encoding_utils

### Look at examples

In [ ]:
class A:
    use_test_setup = False
    subject = 'UTS03'


args = A()
story_names_train, story_names_test = fit_encoding.get_story_names(args)
wordseqs = feature_spaces.get_story_wordseqs(story_names_train)

### Generate examples for questions v3

In [ ]:
seed = 43  # 42, 43
ngrams_examples = []
ngram_size = 10
num_examples_per_story = 1
random.seed(seed)
np.random.seed(seed)
for story_name in story_names_train:
    words_list = wordseqs[story_name].data
    ngrams_list = feature_spaces._get_ngrams_list_from_words_list(
        words_list, ngram_size=ngram_size)[ngram_size + 2:]
    ngrams_examples += np.random.choice(ngrams_list,
                                        num_examples_per_story).tolist()
print('\n'.join(['- ' + ngram for ngram in ngrams_examples]))

### Generate examples for boosted questions v4

In [ ]:
# load top model to boost
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/results_apr1'
r = imodelsx.process_results.get_results_df(results_dir)
for k in ['save_dir', 'save_dir_unique']:
    r[k] = r[k].map(lambda x: x if x.startswith('/home')
                    else x.replace('/mntv1', '/home/chansingh/mntv1'))

args_top = r[
    (r.feature_space.str.contains('qa_embedder')) *
    (r.pc_components == 100) *
    (r.ndelays == 4)
].sort_values(
    by='corrs_tune_pc_mean',
    ascending=False).iloc[0]
print(f'{args_top.feature_space=} {args_top.ndelays=}')
print(f'{args_top.corrs_test_mean=:.3f} {args_top.corrs_tune_pc_mean=:3f}')

In [ ]:
model_params_to_save = joblib.load(
    join(args_top.save_dir_unique, 'model_params.pkl'))

In [ ]:
r = defaultdict(list)
for story_name in tqdm(story_names_train):
    # ngram for 2 trs preceding the current TR
    chunks = wordseqs[story_name].chunks()
    ngrams_list = feature_spaces._get_ngrams_list_from_chunks(
        chunks, num_trs=2)
    ngrams_list = np.array(ngrams_list[10:-5])

    stim_train_delayed, resp_train = fit_encoding.get_data(
        args_top, [story_name])

    preds_test = stim_train_delayed @ model_params_to_save['weights'] + \
        model_params_to_save['bias']

    # calculate correlation at each timepoint
    corrs_time = np.array([np.corrcoef(resp_train[i, :], preds_test[i, :])[0, 1]
                           for i in range(resp_train.shape[0])])
    corrs_time[:10] = 100  # don't pick first 10 TRs
    # get worst 3 idxs
    corrs_worst_idxs = np.argsort(corrs_time)[:3]

    for i in range(3):
        r['story_name'].append(story_name)
        r['corrs'].append(corrs_time[corrs_worst_idxs[i]])
        r['ngram'].append(ngrams_list[corrs_worst_idxs[i]])
        r['tr'].append(corrs_worst_idxs[i])

    joblib.dump(r, 'ngrams_worst.pkl')  # saved as 04_ngrams_boost.pkl

In [32]:
ngrams_boost = pd.DataFrame(joblib.load('04_ngrams_boost.pkl'))

In [44]:
print('\n'.join(['- ' + x for x in ngrams_boost.iloc[1::3]
      ['ngram'].values if len(x.strip()) > 1]))

- on top of that my my wife got pregnant at the time and i realized as that
- off guard but when he got two feet away
- pounds of baby that i'm analyzing her skull and she goes
- therapy he's had shock treatments he's had every combination
- feeling creepy for staring at these strangers and also envious
- by he was also a bipolar and if
- to play and every so often i turn around to look at mike course
- unsettled i sensed i was now exactly where i should be
- and we would he taught me how to chop wood and he bought me um
- me to the town's ramshackle internet cafe so i could check my e-mail
- we were happy we kept all the time in touch with the family
- just lost his wife
- straight at me and almost
- and the snacks that doesn't even start with the clothing so we overpacked
- over blueprints and the building
- in the morning and the streets of delhi were quiet i'd never seen them quiet before they're normally
- education director and i asked him which class he thought i should take and